In [5]:
%matplotlib inline
import numpy as np
import matplotlib.tri as Tri
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from netCDF4 import Dataset as NetCDFFile 
import netCDF4
import datetime as dt
import pandas as pd
from StringIO import StringIO

In [6]:
import re

def extractURLsatellite(fileURL, satName):
    """
    Function to extract the URLs for a specific satellite from the IMOS URLs list
    
    Parameters:
    -----------
    
    - fileURL : IMOS URLs list as a txt file
    - satName : name of the satellite such as JASON-2 JASON-3 
    
    Ouputs:
    ------
    
    - getFiles : list of URLs for the desired satellite
    
    """
    
    getFiles = []
    
    with open(fileURL) as f:
        for line in f:
            if re.search(r"%s"%satName, line):
                changeURL = re.sub('http://data.aodn.org.au', 'http://thredds.aodn.org.au/thredds/dodsC', line)
                getFiles.append(changeURL)
                
    return getFiles

In [7]:
jason2URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'JASON-2')

jason3URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'JASON-3')

saralURL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'SARAL')

sentinel3aURL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'SENTINEL-3A')

cryosat2URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'CRYOSAT-2')

enviURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'ENVISAT')

geosatURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'GEOSAT')

ersURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'ERS-2')

gfoURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'GFO')

topURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'TOPEX')


In [8]:
#Hydrographers pretend coordinates

latmin = -19.9
latmax = -18
lonmin = 150
lonmax = 151.99

if latmin>latmax:
    print('Error wrong definition of min and max lat!!!')

if lonmin>lonmax:
    print('Error wrong definition of min and max lon!!!')


In [51]:
boxLat = []
boxLon = []
boxWh = []
boxT = []
for k in range(50):
    j2 = NetCDFFile(topURL[k])
    lats = j2.variables['LATITUDE'][:]
    lons = j2.variables['LONGITUDE'][:]
    wh = j2.variables['SWH_KU_CAL'][:]
    t = j2.variables['TIME'][:]
    
    # Check if satellite points are in the box
    id1 = lats>latmin 
    id2 = lats<latmax
    id3 = lons>lonmin
    id4 = lons<lonmax
    id1 = id1*1
    id2 = id2*1
    id3 = id3*1
    id4 = id4*1
    tot = id1+id2+id3+id4
    # If this is the case than the sum should be 4
    inside = np.where(tot==4)[:]
    
    if len(inside)>0:
        boxLat.append(lats[inside])
        boxLon.append(lons[inside])
        boxWh.append(wh[inside])
        boxT.append(t[inside])
print len(inside)    

1


In [52]:
for k in range(len(boxLat)):
    if k == 0:
        lat = boxLat[k]
        lon = boxLon[k]
        wh = boxWh[k]
        t = boxT[k]
    else:
        lat = np.append(lat,boxLat[k])
        lon = np.append(lon,boxLon[k])
        wh = np.append(wh,boxWh[k])
        t = np.append(t,boxT[k])

In [53]:
print lats.shape
print lons.shape
print wh.shape
print t.shape

(7800,)
(7800,)
(13692,)
(13692,)


In [54]:
ncs = NetCDFFile(topURL[0])
start = dt.datetime(1950,1,1)
# Get desired time step  
time_var = ncs.variables['TIME']
itime = netCDF4.date2index(start,time_var,select='nearest')
dtime = netCDF4.num2date(time_var[itime],time_var.units)
daystr = dtime.strftime('%Y-%b-%d %H:%M:%S')
print 'Saral record start time:',daystr

end = dt.datetime(2020,2,1)
# Get desired time step  
time_var = ncs.variables['TIME']
itime2 = netCDF4.date2index(end,time_var,select='nearest')
dtime2 = netCDF4.num2date(time_var[itime2],time_var.units)
dayend = dtime2.strftime('%Y-%b-%d %H:%M:%S')
#h=nc.variables['SWH_KU_CAL']
print 'Saral record end time:',dayend

Saral record start time: 1992-Oct-14 08:57:19
Saral record end time: 2002-Sep-03 02:30:33


In [197]:
ncs = NetCDFFile(topURL[0])
ncs.variables['TIME'][0]
dtime2 = netCDF4.num2date(ncs.variables['TIME'][0:150],time_var.units)
print dtime2

[datetime.datetime(1992, 10, 14, 8, 57, 19, 750004)
 datetime.datetime(1992, 10, 14, 8, 57, 20, 820316)
 datetime.datetime(1992, 10, 14, 8, 57, 21, 898434)
 datetime.datetime(1992, 10, 14, 8, 57, 22, 968746)
 datetime.datetime(1992, 10, 14, 8, 57, 24, 46873)
 datetime.datetime(1992, 10, 14, 8, 57, 25, 117185)
 datetime.datetime(1992, 10, 14, 8, 57, 26, 195312)
 datetime.datetime(1992, 10, 14, 8, 57, 27, 265624)
 datetime.datetime(1992, 10, 14, 8, 57, 28, 343752)
 datetime.datetime(1992, 10, 14, 8, 57, 29, 421879)
 datetime.datetime(1992, 10, 14, 8, 57, 30, 492191)
 datetime.datetime(1992, 10, 14, 8, 57, 31, 570308)
 datetime.datetime(1992, 10, 14, 8, 57, 32, 640620)
 datetime.datetime(1992, 10, 14, 8, 57, 33, 718748)
 datetime.datetime(1992, 10, 14, 8, 57, 34, 789060)
 datetime.datetime(1992, 10, 14, 8, 57, 35, 867187)
 datetime.datetime(1992, 10, 14, 8, 57, 36, 937499)
 datetime.datetime(1992, 10, 14, 8, 57, 38, 15626)
 datetime.datetime(1992, 10, 14, 8, 57, 39, 85938)
 datetime.datet

In [163]:
#print jason2URL[0]

In [80]:
#print jason2URL[1]

In [82]:
print min(wh)
print max(wh)

0.041
4.907


In [83]:
ids = np.where(wh>0)[0]
print ids

[   0    1    2 ..., 2415 2416 2417]


In [84]:
whpos = wh[ids]

In [85]:
print max(whpos)

4.907


Time

In [90]:
from netCDF4 import Dataset as NetCDFFile 
import datetime as dt